In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [2]:
data_dir = "Dataset"
train_dir = os.path.join(data_dir, "train")
val_dir = os.path.join(data_dir, "validation")
test_dir = os.path.join(data_dir, "test")

In [3]:
def load_and_preprocess_image(image_path):
    """Load and preprocess an image."""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image = cv2.resize(image, (128, 128))  # Resize
    image = apply_preprocessing(image)
    image = image / 255.0  # Normalize
    return image

In [4]:
def apply_preprocessing(image):
    """Apply multiple preprocessing steps."""
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(gray)
    
    _, thresholded = cv2.threshold(enhanced, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    kernel = np.ones((3, 3), np.uint8)
    morph = cv2.morphologyEx(thresholded, cv2.MORPH_OPEN, kernel)
    
    edges = cv2.Canny(morph, 50, 150)
    
    sharpen_kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(image, -1, sharpen_kernel)
    
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    
    return sharpened


In [5]:
data_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    horizontal_flip=True,
    vertical_flip=True
)

train_data = data_gen.flow_from_directory(
    train_dir, target_size=(128, 128), batch_size=32, class_mode='categorical')
val_data = data_gen.flow_from_directory(
    val_dir, target_size=(128, 128), batch_size=32, class_mode='categorical')
test_data = data_gen.flow_from_directory(
    test_dir, target_size=(128, 128), batch_size=32, class_mode='categorical')

Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [10]:
def build_cnn_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')  # 3 classes: Healthy, Powdery, Rust
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

cnn_model = build_cnn_model()
cnn_model.summary()

# Train Model
history = cnn_model.fit(train_data, validation_data=val_data, epochs=20)


C:\Users\vibha\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.3330 - loss: 1.2447 - val_accuracy: 0.6000 - val_loss: 0.7905
Epoch 2/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 94s 2s/step - accuracy: 0.6286 - loss: 0.7188 - val_accuracy: 0.5667 - val_loss: 0.7916
Epoch 3/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.7509 - loss: 0.6147 - val_accuracy: 0.8500 - val_loss: 0.4096
Epoch 4/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 95s 2s/step - accuracy: 0.8150 - loss: 0.4822 - val_accuracy: 0.7667 - val_loss: 0.5621
Epoch 5/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.8456 - loss: 0.4188 - val_accuracy: 0.7667 - val_loss: 0.7074
Epoch 6/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 97s 2s/step - accuracy: 0.8721 - loss: 0.3310 - val_accuracy: 0.8333 - val_loss: 0.6455
Epoch 7/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 90s 2s/step - accuracy: 0.9007 - loss: 0.2848 - val_accuracy: 0.9333 - val_loss: 0.3982
Epoch 8/20
42/42 ━━━━━━━━━━━━━━━━━━━━ 100s 2s/step - accuracy: 0.9333 - loss: 0.1768 - val_accuracy: 0.8500 - val_los

In [11]:
test_loss, test_acc = cnn_model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.9514 - loss: 0.2103
Test Accuracy: 0.9467


In [12]:
cnn_model.save("grape_leaf_cnn_model.h5")


In [13]:
print(f"Model Accuracy: {test_acc * 100:.2f}%")

Model Accuracy: 94.67%
